In [ ]:
from typing import TypedDict, Annotated, Optional, List, Dict, Any
import operator
from pathlib import Path
import os
import logging
import json
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage
from langgraph.graph import StateGraph, END
from datetime import datetime

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

class CodeGenerationState(TypedDict):
    """State management for code generation process"""
    messages: Annotated[list[AnyMessage], operator.add]
    current_code: Optional[str]
    validation_status: Optional[bool]
    error_messages: Optional[list[str]]
    attempt_number: int

class CodeGenerator:
    """Main class for generating, validating, and correcting code"""
    
    def __init__(self, model, checkpointer, base_output_dir: str):
        self.model = model
        self.base_output_dir = base_output_dir
        self.current_attempt = 0
        self.max_attempts = 15
        
        # Create base output directory with timestamp
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        self.base_output_dir = os.path.join(base_output_dir, f"generation_{timestamp}")
        os.makedirs(self.base_output_dir, exist_ok=True)
        
        # Initialize graph
        graph = StateGraph(CodeGenerationState)
        
        # Add nodes
        graph.add_node("developer", self.developer)
        graph.add_node("validator", self.validator)
        graph.add_node("correction", self.correction)
        
        # Add edges
        graph.add_edge("developer", "validator")
        graph.add_conditional_edges(
            "validator",
            lambda state: self.validator(state)["validation_status"],
            {
                True: END,
                False: "correction"
            }
        )
        graph.add_edge("correction", "validator")
        
        # Set entry point
        graph.set_entry_point("developer")
        self.graph = graph.compile(checkpointer=checkpointer)

    def format_requirements_prompt(self, requirements: Dict[str, Any]) -> str:
        """Format requirements into a development prompt"""
        title = requirements.get('Title', 'Code Generation Task')
        description = requirements.get('Description', '')
        acceptance_criteria = requirements.get('AcceptanceCriteria', [])
        
        return f"""
You are an expert Python developer. Generate production-ready Python code based on these requirements.

REQUIREMENTS SPECIFICATION:
Title: {title}
Description: {description}

Acceptance Criteria:
{chr(10).join(f'- {criterion}' for criterion in acceptance_criteria)}

CODE GENERATION GUIDELINES:

1. Code Structure:
   - Create a complete, self-contained Python script
   - Use proper class and function organization
   - Implement all specified functionality
   - Include all necessary imports
   - Create appropriate classes with clear responsibilities
   - Use dataclasses for configuration management
   - Implement utility functions as needed

2. Code Quality Requirements:
   - Follow PEP 8 standards strictly
   - Use type hints for all functions and methods
   - Include detailed docstrings (Google style)
   - Implement comprehensive error handling
   - Add proper logging throughout
   - Make code modular and maintainable

3. Implementation Requirements:
   - Create robust classes and methods
   - Include all necessary validation
   - Handle edge cases properly
   - Implement batch processing if needed
   - Add progress tracking
   - Save or return results appropriately
   - Use appropriate design patterns

4. Documentation:
   - Add detailed docstrings for all components
   - Include usage examples in docstrings
   - Document assumptions and limitations
   - Add inline comments for complex logic
   - Include type hints for clarity

5. Error Handling:
   - Validate all inputs thoroughly
   - Handle expected error cases
   - Provide clear error messages
   - Implement proper logging
   - Use custom exceptions if needed

Return ONLY the complete, runnable Python code without any markdown or explanations.
"""

    def save_code_attempt(self, code: str, attempt_number: int, status: str = "initial") -> str:
        """Save code attempt to file and return the directory path"""
        attempt_dir = os.path.join(self.base_output_dir, f"attempt_{attempt_number}_{status}")
        os.makedirs(attempt_dir, exist_ok=True)
        
        code_file = os.path.join(attempt_dir, "code.py")
        with open(code_file, 'w') as f:
            f.write(code)
        
        logger.info(f"Saved code attempt {attempt_number} to {code_file}")
        return attempt_dir

    def save_validation_results(self, validation_result: Dict[str, Any], attempt_number: int) -> None:
        """Save validation results to file"""
        validation_file = os.path.join(
            self.base_output_dir, 
            f"attempt_{attempt_number}_validation.json"
        )
        with open(validation_file, 'w') as f:
            json.dump(validation_result, f, indent=2)
        
        logger.info(f"Saved validation results to {validation_file}")

    def developer(self, state: CodeGenerationState) -> Dict[str, Any]:
        """Generate initial code or process corrections"""
        try:
            messages = state['messages']
            requirements = json.loads(messages[0].content)
            
            # Format prompt with requirements
            prompt = self.format_requirements_prompt(requirements)
            messages.append(SystemMessage(content=prompt))
            
            # Generate code
            response = self.model.invoke(messages)
            generated_code = response.content
            
            # Save initial code
            self.current_attempt += 1
            attempt_dir = self.save_code_attempt(generated_code, self.current_attempt)
            
            return {
                'messages': messages,
                'current_code': generated_code,
                'validation_status': None,
                'error_messages': [],
                'attempt_number': self.current_attempt
            }
            
        except Exception as e:
            logger.error(f"Error in developer node: {str(e)}")
            return {
                'messages': messages,
                'error_messages': [str(e)],
                'attempt_number': self.current_attempt
            }

    def validator(self, state: CodeGenerationState) -> Dict[str, Any]:
        """Validate generated code"""
        try:
            messages = state.get("messages", [])
            current_code = state.get("current_code", "")
            
            # Validation prompt
            validation_prompt = """
You are an expert code reviewer. Validate the provided Python code thoroughly.

VALIDATION CRITERIA:
1. Code Completeness:
   - All required functionality implemented
   - Proper error handling included
   - Complete class/function implementations
   - Necessary imports present

2. Code Quality:
   - PEP 8 compliance
   - Proper type hints
   - Comprehensive docstrings
   - Clear organization
   - Error handling
   - Logging implementation

3. Functionality Requirements:
   - All acceptance criteria met
   - Proper input validation
   - Correct implementation logic
   - Appropriate error handling
   - Result handling

Review the code and provide validation results in this JSON format:
{
    "valid": boolean,
    "errors": [
        {
            "category": "missing_feature|implementation_error|quality_issue",
            "description": "Detailed description of the issue",
            "severity": "high|medium|low"
        }
    ],
    "suggestions": [
        {
            "category": "improvement|addition|modification",
            "description": "Detailed suggestion for improvement"
        }
    ]
}
"""
            
            validation_message = SystemMessage(content=validation_prompt)
            messages.append(validation_message)
            messages.append(HumanMessage(content=current_code))
            
            # Get validation feedback
            response = self.model.invoke(messages)
            validation_result = json.loads(response.content)
            
            # Save validation results
            self.save_validation_results(validation_result, state['attempt_number'])
            
            # Save validated code
            self.save_code_attempt(
                current_code, 
                state['attempt_number'],
                f"validated_{'pass' if validation_result['valid'] else 'fail'}"
            )
            
            return {
                'messages': messages,
                'current_code': current_code,
                'validation_status': validation_result['valid'],
                'error_messages': [e['description'] for e in validation_result.get('errors', [])],
                'attempt_number': state['attempt_number']
            }
            
        except Exception as e:
            logger.error(f"Error in validator node: {str(e)}")
            return {
                'messages': messages,
                'validation_status': False,
                'error_messages': [str(e)],
                'attempt_number': state['attempt_number']
            }

    def correction(self, state: CodeGenerationState) -> Dict[str, Any]:
        """Correct code based on validation feedback"""
        try:
            if self.current_attempt >= self.max_attempts:
                logger.error("Maximum correction attempts reached")
                return {
                    'messages': state['messages'],
                    'validation_status': False,
                    'error_messages': ["Maximum correction attempts reached"],
                    'attempt_number': state['attempt_number']
                }
            
            messages = state['messages']
            error_messages = state.get('error_messages', [])
            
            correction_prompt = f"""
You are an expert Python developer. Fix the provided code based on the validation feedback.

VALIDATION ISSUES:
{chr(10).join(f'- {error}' for error in error_messages)}

REQUIREMENTS:
1. Fix all reported issues
2. Maintain code organization
3. Keep existing functionality
4. Follow best practices
5. Ensure code is complete and runnable

Return ONLY the complete, corrected Python code without any markdown or explanations.
"""
            
            messages.append(SystemMessage(content=correction_prompt))
            
            # Generate corrected code
            response = self.model.invoke(messages)
            corrected_code = response.content
            
            # Save correction attempt
            self.current_attempt += 1
            self.save_code_attempt(corrected_code, self.current_attempt, "correction")
            
            return {
                'messages': messages,
                'current_code': corrected_code,
                'validation_status': None,
                'error_messages': [],
                'attempt_number': self.current_attempt
            }
            
        except Exception as e:
            logger.error(f"Error in correction node: {str(e)}")
            return {
                'messages': state['messages'],
                'error_messages': [str(e)],
                'attempt_number': state['attempt_number']
            }

def generate_code(requirements: Dict[str, Any], output_dir: str = "code_generation"):
    """Main function to generate code based on requirements"""
    from langgraph.checkpoint.sqlite import SqliteSaver
    from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace
    
    # Initialize the model
    llm = HuggingFaceEndpoint(
        repo_id="meta-llama/Meta-Llama-3-8B-Instruct",
        task="text-generation",
        max_new_tokens=8000,
        do_sample=False,
        temperature=0.7
    )
    
    model = ChatHuggingFace(llm=llm, verbose=True)
    
    # Create output directory
    os.makedirs(output_dir, exist_ok=True)
    
    # Initialize code generator
    with SqliteSaver.from_conn_string(":memory:") as checkpointer:
        generator = CodeGenerator(model, checkpointer, output_dir)
        
        # Create initial message with requirements
        user_message = HumanMessage(content=json.dumps(requirements))
        
        # Generate and validate code
        thread = {"configurable": {"thread_id": "1"}}
        final_result = None
        
        for event in generator.graph.stream({"messages": [user_message]}, thread):
            final_result = event
            
            # Log progress
            if event.get('error_messages'):
                logger.info(f"Attempt {event['attempt_number']} - Issues found:")
                for error in event['error_messages']:
                    logger.info(f"- {error}")
            
            if event.get('validation_status') == True:
                logger.info(f"Successfully generated valid code on attempt {event['attempt_number']}")
                break
        
        return final_result

if __name__ == "__main__":
    # Example requirements
    requirements = {
        "Title": "Image Preprocessing as per Model",
        "Description": "As a user, I want the system to perform image preprocessing...",
        "AcceptanceCriteria": [
            "Pre-processing should successfully be implemented on images with the following bit representations: 1-bit, 8-bit,16-bit, 24-bit, 32-bit.",
            "The pre-processing system shall be able to resize the input images into the resolution required by the AI model.",
            "The pre-processing system shall be able to normalize the input image data into appropriate pixel datatype required by the AI model.",
            "When pre-processing is successful for the training dataset, the data shall be sent further for batch creation.",
            "When pre-processing is successful for inferencing (prediction) dataset, the data shall be sent for model inferencing."
        ]
    }
    
    # Generate code
    result = generate_code(requirements)
    
    if result and result.get('validation_status'):
        logger.info("Code generation successful!")
        logger.info(f"Check the generated code in the output directory")
    else:
        logger.error("Failed to generate valid code after maximum attempts")

/home/airangers/anaconda3/envs/e/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-02-25 10:59:36,372 - INFO - Saved code attempt 1 to code_generation/generation_20250225_105919/attempt_1_initial/code.py
2025-02-25 10:59:41,047 - ERROR - Error in validator node: Expecting value: line 1 column 1 (char 0)
2025-02-25 10:59:53,282 - ERROR - Error in validator node: Expecting value: line 1 column 1 (char 0)
2025-02-25 10:59:54,090 - INFO - Saved code attempt 2 to code_generation/generation_20250225_105919/attempt_2_correction/code.py
2025-02-25 10:59:54,602 - ERROR - Error in validator node: 422 Client Error: Unprocessable Entity for url: https://api-inference.huggingface.co/models/meta-llama/Meta-Llama-3-8B-Instruct/v1/chat/completions (Request ID: Root=1-67bd5552-4744621d53260827380c11b8;a0428927-c577-4

In [ ]:
from typing import TypedDict, Annotated, Optional, List, Dict, Any
import operator
from pathlib import Path
import os
import logging
import json
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage
from langgraph.graph import StateGraph, END
from datetime import datetime
import re

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

class CodeGenerationState(TypedDict):
    """State management for code generation process"""
    messages: Annotated[list[AnyMessage], operator.add]
    current_code: Optional[str]
    validation_status: Optional[bool]
    error_messages: Optional[list[str]]
    attempt_number: int

class CodeGenerator:
    """Main class for generating, validating, and correcting code"""
    
    def __init__(self, model, checkpointer, base_output_dir: str):
        self.model = model
        self.base_output_dir = base_output_dir
        self.current_attempt = 0
        self.max_attempts = 15
        
        # Create base output directory with timestamp
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        self.base_output_dir = os.path.join(base_output_dir, f"generation_{timestamp}")
        os.makedirs(self.base_output_dir, exist_ok=True)
        
        # Initialize graph
        graph = StateGraph(CodeGenerationState)
        
        # Add nodes
        graph.add_node("developer", self.developer)
        graph.add_node("validator", self.validator)
        graph.add_node("correction", self.correction)
        
        # Add edges
        graph.add_edge("developer", "validator")
        graph.add_conditional_edges(
            "validator",
            lambda state: self.validator(state)["validation_status"],
            {
                True: END,
                False: "correction"
            }
        )
        graph.add_edge("correction", "validator")
        
        # Set entry point
        graph.set_entry_point("developer")
        self.graph = graph.compile(checkpointer=checkpointer)

    def strip_markdown_code_blocks(self, text: str) -> str:
        """
        Strip markdown code block formatting from text
        Removes triple backticks and language identifiers that often surround code generated by LLMs
        """
        # Check if the text starts with triple backticks and remove the first line if it does
        if text.lstrip().startswith("```"):
            lines = text.split('\n')
            # Skip the first line (which has the opening backticks)
            if len(lines) > 1:
                text = '\n'.join(lines[1:])
        
        # Check if the text ends with triple backticks and remove them
        if text.rstrip().endswith("```"):
            lines = text.split('\n')
            # Remove the last line if it only contains backticks
            if lines[-1].strip() == "```":
                text = '\n'.join(lines[:-1])
                
        return text

    def format_requirements_prompt(self, requirements: Dict[str, Any]) -> str:
        """Format requirements into a development prompt"""
        title = requirements.get('Title', 'Code Generation Task')
        description = requirements.get('Description', '')
        acceptance_criteria = requirements.get('AcceptanceCriteria', [])
        
        return f"""
You are an expert Python developer. Generate production-ready Python code based on these requirements.

REQUIREMENTS SPECIFICATION:
Title: {title}
Description: {description}

Acceptance Criteria:
{chr(10).join(f'- {criterion}' for criterion in acceptance_criteria)}

CODE GENERATION GUIDELINES:

1. Code Structure:
   - Create a complete, self-contained Python script
   - Use proper class and function organization
   - Implement all specified functionality
   - Include all necessary imports
   - Create appropriate classes with clear responsibilities
   - Use dataclasses for configuration management
   - Implement utility functions as needed

2. Code Quality Requirements:
   - Follow PEP 8 standards strictly
   - Use type hints for all functions and methods
   - Include detailed docstrings (Google style)
   - Implement comprehensive error handling
   - Add proper logging throughout
   - Make code modular and maintainable

3. Implementation Requirements:
   - Create robust classes and methods
   - Include all necessary validation
   - Handle edge cases properly
   - Implement batch processing if needed
   - Add progress tracking
   - Save or return results appropriately
   - Use appropriate design patterns

4. Documentation:
   - Add detailed docstrings for all components
   - Include usage examples in docstrings
   - Document assumptions and limitations
   - Add inline comments for complex logic
   - Include type hints for clarity

5. Error Handling:
   - Validate all inputs thoroughly
   - Handle expected error cases
   - Provide clear error messages
   - Implement proper logging
   - Use custom exceptions if needed

Return ONLY the complete, runnable Python code without any markdown or explanations.
"""

    def save_code_attempt(self, code: str, attempt_number: int, status: str = "initial") -> str:
        """Save code attempt to file and return the directory path"""
        attempt_dir = os.path.join(self.base_output_dir, f"attempt_{attempt_number}_{status}")
        os.makedirs(attempt_dir, exist_ok=True)
        
        code_file = os.path.join(attempt_dir, "code.py")
        with open(code_file, 'w') as f:
            f.write(code)
        
        logger.info(f"Saved code attempt {attempt_number} to {code_file}")
        return attempt_dir

    def save_validation_results(self, validation_result: Dict[str, Any], attempt_number: int) -> None:
        """Save validation results to file"""
        validation_file = os.path.join(
            self.base_output_dir, 
            f"attempt_{attempt_number}_validation.json"
        )
        with open(validation_file, 'w') as f:
            json.dump(validation_result, f, indent=2)
        
        logger.info(f"Saved validation results to {validation_file}")

    def developer(self, state: CodeGenerationState) -> Dict[str, Any]:
        """Generate initial code or process corrections"""
        try:
            messages = state['messages']
            requirements = json.loads(messages[0].content)
            
            # Format prompt with requirements
            prompt = self.format_requirements_prompt(requirements)
            messages.append(SystemMessage(content=prompt))
            
            # Generate code
            response = self.model.invoke(messages)
            generated_code = response.content
            
            # Strip markdown code blocks if present
            generated_code = self.strip_markdown_code_blocks(generated_code)
            
            # Save initial code
            self.current_attempt += 1
            attempt_dir = self.save_code_attempt(generated_code, self.current_attempt)
            
            return {
                'messages': messages,
                'current_code': generated_code,
                'validation_status': None,
                'error_messages': [],
                'attempt_number': self.current_attempt
            }
            
        except Exception as e:
            logger.error(f"Error in developer node: {str(e)}")
            return {
                'messages': messages,
                'error_messages': [str(e)],
                'attempt_number': self.current_attempt
            }

    def validator(self, state: CodeGenerationState) -> Dict[str, Any]:
        """Validate generated code"""
        try:
            messages = state.get("messages", [])
            current_code = state.get("current_code", "")
            
            # Strip markdown code blocks if present
            current_code = self.strip_markdown_code_blocks(current_code)
            
            # Validation prompt
            validation_prompt = """
You are an expert code reviewer. Validate the provided Python code thoroughly.

VALIDATION CRITERIA:
1. Code Completeness:
   - All required functionality implemented
   - Proper error handling included
   - Complete class/function implementations
   - Necessary imports present

2. Code Quality:
   - PEP 8 compliance
   - Proper type hints
   - Comprehensive docstrings
   - Clear organization
   - Error handling
   - Logging implementation

3. Functionality Requirements:
   - All acceptance criteria met
   - Proper input validation
   - Correct implementation logic
   - Appropriate error handling
   - Result handling

Review the code and provide validation results in this JSON format:
{
    "valid": boolean,
    "errors": [
        {
            "category": "missing_feature|implementation_error|quality_issue",
            "description": "Detailed description of the issue",
            "severity": "high|medium|low"
        }
    ],
    "suggestions": [
        {
            "category": "improvement|addition|modification",
            "description": "Detailed suggestion for improvement"
        }
    ]
}
"""
            
            validation_message = SystemMessage(content=validation_prompt)
            messages.append(validation_message)
            messages.append(HumanMessage(content=current_code))
            
            # Get validation feedback
            response = self.model.invoke(messages)
            validation_result = json.loads(response.content)
            
            # Save validation results
            self.save_validation_results(validation_result, state['attempt_number'])
            
            # Save validated code
            self.save_code_attempt(
                current_code, 
                state['attempt_number'],
                f"validated_{'pass' if validation_result['valid'] else 'fail'}"
            )
            
            # Update current_code in state with the stripped version
            state['current_code'] = current_code
            
            return {
                'messages': messages,
                'current_code': current_code,
                'validation_status': validation_result['valid'],
                'error_messages': [e['description'] for e in validation_result.get('errors', [])],
                'attempt_number': state['attempt_number']
            }
            
        except Exception as e:
            logger.error(f"Error in validator node: {str(e)}")
            return {
                'messages': messages,
                'validation_status': False,
                'error_messages': [str(e)],
                'attempt_number': state['attempt_number']
            }

    def correction(self, state: CodeGenerationState) -> Dict[str, Any]:
        """Correct code based on validation feedback"""
        try:
            if self.current_attempt >= self.max_attempts:
                logger.error("Maximum correction attempts reached")
                return {
                    'messages': state['messages'],
                    'validation_status': False,
                    'error_messages': ["Maximum correction attempts reached"],
                    'attempt_number': state['attempt_number']
                }
            
            messages = state['messages']
            error_messages = state.get('error_messages', [])
            current_code = state.get('current_code', '')
            
            # Strip markdown code blocks if present
            current_code = self.strip_markdown_code_blocks(current_code)
            
            correction_prompt = f"""
You are an expert Python developer. Fix the provided code based on the validation feedback.

VALIDATION ISSUES:
{chr(10).join(f'- {error}' for error in error_messages)}

REQUIREMENTS:
1. Fix all reported issues
2. Maintain code organization
3. Keep existing functionality
4. Follow best practices
5. Ensure code is complete and runnable

Return ONLY the complete, corrected Python code without any markdown or explanations.
"""
            
            messages.append(SystemMessage(content=correction_prompt))
            
            # Generate corrected code
            response = self.model.invoke(messages)
            corrected_code = response.content
            
            # Strip markdown code blocks if present
            corrected_code = self.strip_markdown_code_blocks(corrected_code)
            
            # Save correction attempt
            self.current_attempt += 1
            self.save_code_attempt(corrected_code, self.current_attempt, "correction")
            
            return {
                'messages': messages,
                'current_code': corrected_code,
                'validation_status': None,
                'error_messages': [],
                'attempt_number': self.current_attempt
            }
            
        except Exception as e:
            logger.error(f"Error in correction node: {str(e)}")
            return {
                'messages': state['messages'],
                'error_messages': [str(e)],
                'attempt_number': state['attempt_number']
            }

def generate_code(requirements: Dict[str, Any], output_dir: str = "code_generation"):
    """Main function to generate code based on requirements"""
    from langgraph.checkpoint.sqlite import SqliteSaver
    from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace
    
    # Initialize the model
    llm = HuggingFaceEndpoint(
        repo_id="meta-llama/Meta-Llama-3-8B-Instruct",
        task="text-generation",
        max_new_tokens=8000,
        do_sample=False,
        temperature=0.7
    )
    
    model = ChatHuggingFace(llm=llm, verbose=True)
    
    # Create output directory
    os.makedirs(output_dir, exist_ok=True)
    
    # Initialize code generator
    with SqliteSaver.from_conn_string(":memory:") as checkpointer:
        generator = CodeGenerator(model, checkpointer, output_dir)
        
        # Create initial message with requirements
        user_message = HumanMessage(content=json.dumps(requirements))
        
        # Generate and validate code
        thread = {"configurable": {"thread_id": "1"}}
        final_result = None
        
        for event in generator.graph.stream({"messages": [user_message]}, thread):
            final_result = event
            
            # Log progress
            if event.get('error_messages'):
                logger.info(f"Attempt {event['attempt_number']} - Issues found:")
                for error in event['error_messages']:
                    logger.info(f"- {error}")
            
            if event.get('validation_status') == True:
                logger.info(f"Successfully generated valid code on attempt {event['attempt_number']}")
                break
        
        return final_result

if __name__ == "__main__":
    # Example requirements
    requirements = {
        "Title": "Image Preprocessing as per Model",
        "Description": "As a user, I want the system to perform image preprocessing...",
        "AcceptanceCriteria": [
            "Pre-processing should successfully be implemented on images with the following bit representations: 1-bit, 8-bit,16-bit, 24-bit, 32-bit.",
            "The pre-processing system shall be able to resize the input images into the resolution required by the AI model.",
            "The pre-processing system shall be able to normalize the input image data into appropriate pixel datatype required by the AI model.",
            "When pre-processing is successful for the training dataset, the data shall be sent further for batch creation.",
            "When pre-processing is successful for inferencing (prediction) dataset, the data shall be sent for model inferencing."
        ]
    }
    
    # Generate code
    result = generate_code(requirements)
    
    if result and result.get('validation_status'):
        logger.info("Code generation successful!")
        logger.info(f"Check the generated code in the output directory")
    else:
        logger.error("Failed to generate valid code after maximum attempts")

/home/airangers/anaconda3/envs/e/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-02-25 11:07:16,800 - INFO - Saved code attempt 1 to code_generation/generation_20250225_110716/attempt_1_initial/code.py
2025-02-25 11:08:03,829 - ERROR - Error in validator node: Expecting value: line 1 column 5 (char 4)
2025-02-25 11:08:14,431 - ERROR - Error in validator node: Expecting value: line 1 column 13 (char 12)
2025-02-25 11:08:36,671 - INFO - Saved code attempt 2 to code_generation/generation_20250225_110716/attempt_2_correction/code.py
2025-02-25 11:08:37,095 - ERROR - Error in validator node: 422 Client Error: Unprocessable Entity for url: https://api-inference.huggingface.co/models/meta-llama/Meta-Llama-3-8B-Instruct/v1/chat/completions (Request ID: Root=1-67bd575c-46d4246a3679634d487b1808;9c80c6da-0fc4